# Building a simple autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import optimizers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Standard autoencoder

Design the network. A single dense layer for the encoder and decoder needs an encoding dimension of around 100. Having more layers allows a smaller encoding dimension, although convergence during learning starts to become unstable.

In [3]:
encoding_dim = 16
input_reg = keras.Input(shape=(vec_length,))
encoded = layers.Dense(vec_length, activation='relu')(input_reg)
encoded = layers.Dense(70, activation='relu')(encoded)
encoded = layers.Dense(30, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoder = keras.Model(input_reg, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoded = encoded_input
decoded = layers.Dense(30, activation='relu')(decoded)
decoded = layers.Dense(70, activation='relu')(decoded)
decoded = layers.Dense(vec_length, activation='sigmoid')(decoded)
decoder = keras.Model(encoded_input, decoded)

autoencoder = keras.Model(input_reg, decoder(encoder(input_reg)))

Do some learning. For shallow networks, almost any optimizer and loss work. For deeper networks, poor choices almost certainly yield failed convergence on the loss function. Even good choices yield convergence on the loss function only on some runs.

In [4]:
opt = optimizers.Adamax(learning_rate=0.003)
autoencoder.compile(optimizer=opt, loss='binary_crossentropy')
autoencoder.fit(train_vecs, train_vecs,
                epochs=100,
                batch_size=8,
                shuffle=True,
                validation_data=(val_vecs, val_vecs))

Epoch 1/100
1250/1250 [==============================] - 1s 800us/step - loss: 0.1820 - val_loss: 0.1654
Epoch 2/100
1250/1250 [==============================] - 1s 739us/step - loss: 0.1537 - val_loss: 0.1478
Epoch 3/100
1250/1250 [==============================] - 1s 735us/step - loss: 0.1415 - val_loss: 0.1369
Epoch 4/100
1250/1250 [==============================] - 1s 749us/step - loss: 0.1305 - val_loss: 0.1312
Epoch 5/100
1250/1250 [==============================] - 1s 750us/step - loss: 0.1251 - val_loss: 0.1271
Epoch 6/100
1250/1250 [==============================] - 1s 751us/step - loss: 0.1210 - val_loss: 0.1225
Epoch 7/100
1250/1250 [==============================] - 1s 739us/step - loss: 0.1159 - val_loss: 0.1186
Epoch 8/100
1250/1250 [==============================] - 1s 751us/step - loss: 0.1126 - val_loss: 0.1162
Epoch 9/100
1250/1250 [==============================] - 1s 753us/step - loss: 0.1098 - val_loss: 0.1144
Epoch 10/100
1250/1250 [==============================]

1250/1250 [==============================] - 1s 757us/step - loss: 0.0132 - val_loss: 0.0192
Epoch 79/100
1250/1250 [==============================] - 1s 721us/step - loss: 0.0125 - val_loss: 0.0182
Epoch 80/100
1250/1250 [==============================] - 1s 736us/step - loss: 0.0115 - val_loss: 0.0151
Epoch 81/100
1250/1250 [==============================] - 1s 756us/step - loss: 0.0114 - val_loss: 0.0146
Epoch 82/100
1250/1250 [==============================] - 1s 760us/step - loss: 0.0104 - val_loss: 0.0144
Epoch 83/100
1250/1250 [==============================] - 1s 742us/step - loss: 0.0104 - val_loss: 0.0160
Epoch 84/100
1250/1250 [==============================] - 1s 784us/step - loss: 0.0095 - val_loss: 0.0154
Epoch 85/100
1250/1250 [==============================] - 1s 778us/step - loss: 0.0094 - val_loss: 0.0122
Epoch 86/100
1250/1250 [==============================] - 1s 686us/step - loss: 0.0087 - val_loss: 0.0138
Epoch 87/100
1250/1250 [==============================] - 1

Evaluation

In [5]:
decoded_regs = autoencoder.predict(test_vecs)
print(test_strs[:10])
recovered = [v.recover(x) for x in decoded_regs]
print(recovered[:10])
acc = np.sum([x == y for x, y in zip(recovered, test_strs)]) / len(test_strs)
print('overall accuracy = ' + str(acc))
print('by digit accuracy:')
for i_digit in range(len(test_strs[0])):
    acc = np.sum([x[i_digit] == y[i_digit] for x, y in zip(recovered, test_strs)]) / len(test_strs)
    print('    ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
overall accuracy = 0.95
by digit accuracy:
    0.996
    0.998
    1.0
    0.995
    0.979
    0.995
    0.986
